In [35]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np

# Data Processing

In [37]:
# Load the data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [38]:
# Change the labels to be hot encoded
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [36]:
# Define a decaying learning rate schedule
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003        
    return lrate

In [ ]:
# Set hyper parameters
weight_decay = 1e-4
batch_size = 64

# Define a consistent optimiser
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)

# Support Vector Machine

In [ ]:
# Jialu

# Multi Layer Perceptron
##### Create a Neural Network with three hidden layers, batch normalisation, and dropout.

In [41]:
mlp = Sequential()

mlp.add(Dense(1024, input_shape=x_train.shape[1:]))
mlp.add(Activation('elu'))
mlp.add(BatchNormalization())
mlp.add(Dropout(0.25))

mlp.add(Dense(256))
mlp.add(Activation('elu'))
mlp.add(BatchNormalization())
mlp.add(Dropout(0.25))

mlp.add(Dense(64))
mlp.add(Activation('elu'))
mlp.add(BatchNormalization())
mlp.add(Dropout(0.25))

mlp.add(Flatten())
mlp.add(Dense(10))
mlp.add(Activation('softmax'))

mlp.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 32, 32, 1024)      4096      
_________________________________________________________________
activation_22 (Activation)   (None, 32, 32, 1024)      0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 32, 32, 1024)      4096      
_________________________________________________________________
dropout_16 (Dropout)         (None, 32, 32, 1024)      0         
_________________________________________________________________
dense_25 (Dense)             (None, 32, 32, 256)       262400    
_________________________________________________________________
activation_23 (Activation)   (None, 32, 32, 256)       0         
_________________________________________________________________
batch_normalization_17 (Batc (None, 32, 32, 256)       1024      
__________

In [42]:
datagen_mlp = ImageDataGenerator()
datagen_mlp.fit(x_train)

mlp.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
mlp.fit_generator(datagen_mlp.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size, epochs=20,\
                    verbose=1, validation_data=(x_test,y_test), callbacks=[LearningRateScheduler(lr_schedule)])

Epoch 1/20
781/781 [==============================] - 63s 80ms/step - loss: 9.9650 - acc: 0.3450 - val_loss: 9.5631 - val_acc: 0.3870
Epoch 2/20
781/781 [==============================] - 61s 79ms/step - loss: 9.5149 - acc: 0.3900 - val_loss: 9.3319 - val_acc: 0.4023
Epoch 3/20
781/781 [==============================] - 61s 79ms/step - loss: 9.1826 - acc: 0.4136 - val_loss: 9.5872 - val_acc: 0.3882
Epoch 4/20
781/781 [==============================] - 61s 78ms/step - loss: 9.1212 - acc: 0.4196 - val_loss: 9.1786 - val_acc: 0.4150
Epoch 5/20
781/781 [==============================] - 62s 79ms/step - loss: 8.8913 - acc: 0.4338 - val_loss: 9.0061 - val_acc: 0.4239
Epoch 6/20
781/781 [==============================] - 61s 79ms/step - loss: 8.7279 - acc: 0.4442 - val_loss: 8.8194 - val_acc: 0.4377
Epoch 7/20
781/781 [==============================] - 62s 79ms/step - loss: 8.7012 - acc: 0.4474 - val_loss: 8.9921 - val_acc: 0.4294
Epoch 8/20
781/781 [==============================] - 61s 79ms

# Convolutional Neural Network
##### Create a convolutional neural network with 3 convolutional layers, batch normalisation, dropout and max pooling.

In [5]:
# This model is definitely plagiarised, so will need to edit it somehow, I can do that tomorrow.

model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
__________

In [5]:
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

In [7]:
#training

model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=125,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])

Epoch 1/125
781/781 [==============================] - 15s 20ms/step - loss: 1.8757 - acc: 0.4318 - val_loss: 1.5947 - val_acc: 0.5426
Epoch 2/125
781/781 [==============================] - 14s 18ms/step - loss: 1.2779 - acc: 0.5947 - val_loss: 1.0920 - val_acc: 0.6763
Epoch 3/125
781/781 [==============================] - 14s 18ms/step - loss: 1.0802 - acc: 0.6609 - val_loss: 0.9899 - val_acc: 0.6981
Epoch 4/125
781/781 [==============================] - 14s 18ms/step - loss: 0.9764 - acc: 0.6972 - val_loss: 0.9168 - val_acc: 0.7287
Epoch 5/125
781/781 [==============================] - 14s 18ms/step - loss: 0.9108 - acc: 0.7203 - val_loss: 0.8943 - val_acc: 0.7372
Epoch 6/125
781/781 [==============================] - 14s 18ms/step - loss: 0.8701 - acc: 0.7337 - val_loss: 1.0229 - val_acc: 0.7207
Epoch 7/125
781/781 [==============================] - 14s 18ms/step - loss: 0.8324 - acc: 0.7494 - val_loss: 0.9114 - val_acc: 0.7432
Epoch 8/125
781/781 [==============================] - 

781/781 [==============================] - 14s 18ms/step - loss: 0.5916 - acc: 0.8468 - val_loss: 0.5695 - val_acc: 0.8631
Epoch 61/125
781/781 [==============================] - 14s 17ms/step - loss: 0.5851 - acc: 0.8513 - val_loss: 0.6129 - val_acc: 0.8518
Epoch 62/125
781/781 [==============================] - 14s 18ms/step - loss: 0.5904 - acc: 0.8486 - val_loss: 0.5696 - val_acc: 0.8631
Epoch 63/125
781/781 [==============================] - 14s 18ms/step - loss: 0.5857 - acc: 0.8498 - val_loss: 0.7164 - val_acc: 0.8167
Epoch 64/125
781/781 [==============================] - 14s 17ms/step - loss: 0.5861 - acc: 0.8506 - val_loss: 0.5876 - val_acc: 0.8576
Epoch 65/125
781/781 [==============================] - 14s 18ms/step - loss: 0.5837 - acc: 0.8517 - val_loss: 0.5968 - val_acc: 0.8548
Epoch 66/125
781/781 [==============================] - 14s 17ms/step - loss: 0.5834 - acc: 0.8527 - val_loss: 0.6136 - val_acc: 0.8509
Epoch 67/125
781/781 [==============================] - 14s 1

In [9]:
#save to disk
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model.h5') 
 
#testing
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

10000/10000 [==============================] - 1s 101us/step

Test result: 88.090 loss: 0.484


# Resnet50

In [32]:
from keras.applications import ResNet50
from keras.layers import GlobalAveragePooling2D
from keras.models import Model
import scipy

In [34]:
x_train_new = np.zeros(shape=(x_train.shape[0], 224, 224, 3))
for idx in xrange(X_train.shape[0]):
    X_train_new[idx] = X_train[idx].reshape(224, 224, 3)

MemoryError: 

In [7]:
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024)(x)
x = Activation('elu')(x)
predictions = Dense(10, activation='softmax')(x)

resnet_model = Model(inputs=base_model.input, outputs=predictions)

resnet_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [8]:
for layer in base_model.layers: 
    layer.trainable = False

In [9]:
#training
batch_size = 64

opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
resnet_model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

In [ ]:
resnet_model.fit_generator(datagen.flow(x_train_new, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=10,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])

Epoch 1/10
